# Перевести датасет с когнитивными искажениями
https://github.com/psytechlab/empathy_dataset_transfer/issues/33

In [30]:
%load_ext autoreload
%autoreload 2

%precision 3

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


'%.3f'

In [31]:
import pandas as pd

In [32]:
df = pd.read_csv("../Annotated_data.csv")
df.loc[df['Distorted part'].isna(), 'Distorted part'] = ''

In [33]:
df.head()

,Id_Number,Patient Question,Distorted part,Dominant Distortion,Secondary Distortion (Optional)
0,4500,"Hello, I have a beautiful,smart,outgoing and a...",The voice are always fimilar (someone she know...,Personalization,NaN
1,4501,Since I was about 16 years old I’ve had these ...,I feel trapped inside my disgusting self and l...,Labeling,Emotional Reasoning
2,4502,So I’ve been dating on and off this guy for a...,,No Distortion,NaN
3,4503,My parents got divorced in 2004. My mother has...,,No Distortion,NaN
4,4504,I don’t really know how to explain the situati...,I refused to go because I didn’t know if it wa...,Fortune-telling,Emotional Reasoning


In [34]:
def split_distorted_by_max_matches(question, distorted):
    q_lower = question.lower()
    d = distorted.strip()
    i = 0
    segments = []
    while i < len(d):
        best = ""
        for j in range(i+1, len(d)+1):
            sub = d[i:j]
            if sub.lower() in q_lower:
                best = sub
            else:
                break
        if best:
            segments.append(best)
            i += len(best)
        else:
            i += 1 
    return [seg.strip() for seg in segments if seg.strip()]


In [35]:
parts = df['Distorted part'].tolist()
qus = df['Patient Question'].tolist()
new_parts = []
i = 0
inc = []
for q, p in zip(qus, parts):
	if p in q:
		i += 1
		new_parts.append([p])
	else:
		rr = split_distorted_by_max_matches(q, p)
		good = True
		for t in rr:
			if t not in q:
				print(q)
				print()
				inc.append(q)
				good = False
				break
		if good:
			new_parts.append(rr)
		else:
			new_parts.append([])

From a grandmother in Canada: My grandchildren ages 7 and 8 have been telling me for almost 2 years that they are afraid of mommy, my Grandson age 7 started soiling his pants, so I took him to the doctors and he said he’s afraid of mommy and she smacks them quite a bit in the head, I got the CPS involved and they sent the children to counselling. The counsellor seen the children one time and informed the children’s father the children are lying and are making stories up to get mommy and daddy back together.

Hi, Doctor, I think I’m continued dive in the depression. I can’t feel very well. When I trying to get rid of my bad thoughts which is continuously coming to my mind –I can’t. When I trying to concentrate on my work I can’t. Every time I think about ” what happened next or what if I can’t do anything in my life.” I literally struggled to live a normal Happy life. 'Please help me to overcome my meaningless fear and the big one of the big problems for me is I’m not very Happy with my

In [36]:
df['restored_distorted_part'] = new_parts
df.drop(df[df['Patient Question'].isin(inc)].index, inplace=True)

In [38]:
df.head()

,Id_Number,Patient Question,Distorted part,Dominant Distortion,Secondary Distortion (Optional),restored_distorted_part
0,4500,"Hello, I have a beautiful,smart,outgoing and a...",The voice are always fimilar (someone she know...,Personalization,NaN,[The voice are always fimilar (someone she kno...
1,4501,Since I was about 16 years old I’ve had these ...,I feel trapped inside my disgusting self and l...,Labeling,Emotional Reasoning,[I feel trapped inside my disgusting self and ...
2,4502,So I’ve been dating on and off this guy for a...,,No Distortion,NaN,[]
3,4503,My parents got divorced in 2004. My mother has...,,No Distortion,NaN,[]
4,4504,I don’t really know how to explain the situati...,I refused to go because I didn’t know if it wa...,Fortune-telling,Emotional Reasoning,[I refused to go because I didn’t know if it w...


In [39]:
df.shape

(2527, 6)

In [43]:
df.to_csv("annotated_data_restored.csv", index=False, sep="|")